# BEST 2019 Final examination

![](https://github.com/glouppe/info8010-deep-learning/blob/master/tutorials/images/style_transfer.png?raw=1)

## Neural Style Transfer

The goal of your project is to reimplement the algorithm proposed in: [A Neural Algorithm of Artistic Style](https://arxiv.org/pdf/1508.06576.pdf)

We expect you to come up with a working implementation of the original algorithm that is based on the following steps.

This will correspond to your **minimum viable product!**.

- Given any natural and stylized images, the goal is to produce a new image which keeps the content of the first one and applies the style of the latter.

- We expect you to work with a pretrained neural network (e.g. VGG16, ResNet...) which will serve as a feature extractor: to do so you will have to come up with your own forward() function.

- You will have to reimplement the different losses presented in the paper.

- Produce at least one good-looking sample for the content image you were given.

**BONUS:**

- Play with the different layers used for extracting the features.

- Investigate how to balance the preservation of the content with respect to the transferred style.

- Come up with a way that extends the algorithm to the use of multiple styles.

- Investigate the differences between starting with a pretrained network and one which is randomly initialized.



## Non-exhaustive summary of [A Neural Algorithm of Artistic Style](https://arxiv.org/pdf/1508.06576.pdf)

The authors propose to cast the style transfer problem as an optimization procedure over the pixels of the target image. To do so they define a double loss composed of a content loss and style loss:

- The content loss is mathematically defined as the mean squared error (MSE) between the two feature maps (target and content images) over the layers. 
- The style loss can be computed with the MSE between the gram matrices of the vectorized feature maps (one vector per channel). 

The two losses are then combined and minimized with gradient descent.

In [0]:
from torchvision import models
from torchvision import transforms
from PIL import Image
import argparse
import torch
import torchvision
import torch.nn as nn
import numpy as np
from types import SimpleNamespace

### Experimental Setting

In [0]:
config = SimpleNamespace()
config.content = '1.jpg'
config.style = 'style.jpg'
config.style2 = 'box.jpg'
config.max_size = 400
config.total_step = 2000
config.log_step = 10
config.sample_step = 200
config.style_weight = 100
config.lr = .003

In [0]:
# Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Architecture

Define your favourite neural network which will serve as [features extractor](https://pytorch.org/docs/stable/torchvision/models.html).

Redefine your forward pass for computing the features from the network.


In [0]:
class PretrainedNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(PretrainedNet, self).__init__()
        self.select = [0, 5, 7, 23, 9]
        self.pretrainedNet = models.vgg19(pretrained=True).to(device)
        
    def forward(self, x):
        """Extract multiple (5 is good) convolutional feature maps."""
        features = []
        output = x
        for layer_ind in range(len(self.pretrainedNet.features)):
          output = self.pretrainedNet.features[layer_ind](output)
          if layer_ind in self.select:
            features.append(output)
        return features

### Image loader

In [0]:
def load_image(image_path, transform=None, max_size=None, shape=None):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)
    
    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.ANTIALIAS)
    
    if shape:
        image = image.resize(shape, Image.LANCZOS)
    
    if transform:
        image = transform(image).unsqueeze(0)
    
    return image.to(device)

The pytorchvision pretrained models are trained on ImageNet where images are normalized by `mean=[0.485, 0.456, 0.406]` and `std=[0.229, 0.224, 0.225]`. We use the same normalization statistics here.

In [0]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                                                                            std=(0.229, 0.224, 0.225))])

Load content and style images.
Make the style image same size as the content image.


In [0]:
content = load_image(config.content, transform, max_size=config.max_size)
style = load_image(config.style, transform, shape=[content.size(2), content.size(3)])
style2 = load_image(config.style2, transform, shape=[content.size(2), content.size(3)])

### Optimization

Initialize a target image with the content image

In [0]:
target = content.clone().requires_grad_(True)

In [0]:
net = PretrainedNet().eval()
optimizer = torch.optim.Adam([target], lr=0.1)
content_criteria = nn.MSELoss()

The main training loop of the algorithm: we separately deal with the content and style losses (be careful to vectorize your feature maps and follow the formulas presented in the paper).

In [156]:
for step in range(config.total_step):
    # Extract multiple(5) feature maps
    target_features = net.forward(target)
    content_features = net.forward(content) 
    style_features1 = net.forward(style)
    style_features2 = net.forward(style2)
    

    style_loss = 0
    content_loss = 0

    for f1, f2, f3,f4 in zip(target_features, content_features, style_features1,style_features2 ):
        # Compute content loss with target and content images
        content_loss += content_criteria(f1, f2)
        #print(content_loss)

        # Reshape convolutional feature maps
        _, c, h, w = f1.size()
        f1 = f1.reshape(c, h*w).to(device)
        f3 = f3.reshape(c, h*w).to(device)
        f4 = f3.reshape(c, h*w).to(device)

        # Compute gram matrix
        f1 = torch.mm(f1, f1.t())
        f3 = torch.mm(f3, f3.t())
        f4 = torch.mm(f4, f4.t())

        # Compute style loss with target and style images
        style_loss += (1/(len(target_features)*len(content_features))**2)*content_criteria(f1,f3) + (1/(len(target_features)*len(content_features))**2)*content_criteria(f1,f4)
    
    # Compute total loss, backprop and optimize (4 lines of code in total)
    loss = style_loss + len(target_features)*len(content_features)**2*content_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Output
    if (step+1) % config.log_step == 0:
        # Change this print into a log with tensorboardx
        print ('Step [{}/{}], Content Loss: {:.4f}, Style Loss: {:.4f}' 
               .format(step+1, config.total_step, content_loss.item(), style_loss.item()))

    if (step+1) % 50 == 0:
        # Save the generated image (you can also change it to see it with tensorboardx)
        denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
        img = target.clone().squeeze()
        img = denorm(img).clamp_(0, 1)
        torchvision.utils.save_image(img, 'output-{}.png'.format(step+1))

Step [10/2000], Content Loss: 49.8202, Style Loss: 21865538.0000
Step [20/2000], Content Loss: 54.5206, Style Loss: 11413899.0000
Step [30/2000], Content Loss: 56.5388, Style Loss: 6625276.5000
Step [40/2000], Content Loss: 57.5198, Style Loss: 4324963.5000
Step [50/2000], Content Loss: 58.2117, Style Loss: 3129509.0000
Step [60/2000], Content Loss: 58.9963, Style Loss: 2431456.2500
Step [70/2000], Content Loss: 59.6289, Style Loss: 1982983.0000
Step [80/2000], Content Loss: 60.1768, Style Loss: 1671017.8750
Step [90/2000], Content Loss: 60.5980, Style Loss: 1438556.6250
Step [100/2000], Content Loss: 60.9913, Style Loss: 1257589.8750
Step [110/2000], Content Loss: 61.3521, Style Loss: 1110649.8750
Step [120/2000], Content Loss: 61.6712, Style Loss: 988609.3750
Step [130/2000], Content Loss: 61.9644, Style Loss: 885282.0625
Step [140/2000], Content Loss: 62.2082, Style Loss: 797097.0000
Step [150/2000], Content Loss: 62.4270, Style Loss: 721104.4375
Step [160/2000], Content Loss: 62.63

## Bonus

<div class="alert alert-danger">
<b>EXERCISE</b>:


Play with the different layers used for extracting the features.


</div>

<div class="alert alert-danger">
<b>EXERCISE</b>:
    
Investigate how to balance the preservation of the content with respect to the transferred style.


</div>

<div class="alert alert-danger">
<b>EXERCISE</b>:

Come up with a way that extends the algorithm to the use of multiple styles.

</div>

<div class="alert alert-danger">
<b>EXERCISE</b>:

 Investigate the differences between starting with a pretrained network and one which is randomly initialized.


</div>